In [1]:
# This should say you're using a GPU.
# If you aren't using a GPU, go to "Runtime",
# then select "change runtime type" and click
# T4 GPU.

!nvidia-smi

Sun Oct 22 03:13:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/WAT-ai/kernel_tuner

Cloning into 'kernel_tuner'...
remote: Enumerating objects: 9199, done.
remote: Counting objects: 100% (1570/1570), done.
remote: Compressing objects: 100% (373/373), done.
remote: Total 9199 (delta 1338), reused 1316 (delta 1194), pack-reused 7629
Receiving objects: 100% (9199/9199), 4.25 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (6579/6579), done.


In [3]:
pip install kernel_tuner[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=a0a625a19b96dee116909ffe20ed6bae878f31b4f1c6bddadab02464fcdaa378
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24058 sha256=8f8778825008f2947675ca75cbdb18a3083cd

### Demo: Enhancing Convolution Operations with Correctness Verification

In the context of computational operations utilizing convolution kernels, this example operates on a framework that largely mirrors a standard Convolution example. However, a nuanced distinction emerges in the strategic utilization of a naive kernel to establish a reference output. This output subsequently assumes a pivotal role, serving as a benchmark to verify the correctness of each kernel before its performance is benchmarked.

Here's a breakdown of the process without code examples:

1. **Kernel and Data Initialization**:
    - The kernel code is fetched from a predefined file and various data, including filter sizes and problem sizes, is initialized.
    - Input data and filters are set up and computational arguments are defined to facilitate subsequent operations.
  
2. **Parameter Tuning Setup**:
    - A structured collection of tuning parameters is configured, comprising diverse block and tile sizes.
    - Options to utilize or omit padding and the read-only cache are incorporated, providing versatile tuning capabilities.

3. **Reference Output Generation**:
    - A naive convolution kernel is executed, wielding pre-specified parameters, to generate a reference output.
    - This output is crucial as it functions as a benchmark for accuracy in subsequent operations.

4. **Verification and Kernel Tuning**:
    - An 'answer' list is set up, which marks input data as `None` and utilizes non-`None` data for verification purposes.
    - The `tune_kernel` function is invoked to tune the kernel, parallely cross-verifying each kernel in the parameter space against the pre-established reference output.
   
This methodology pivots around utilizing a naive kernel to derive a reference output and subsequently leveraging it for correctness checks during convolution operations, all before performance benchmarking occurs. This ensures computational reliability as it guarantees that only kernels which have been validated for correctness are subjected to the tuning and optimization phase.

In [ ]:
import numpy
import kernel_tuner
from collections import OrderedDict
from kernel_tuner.strategies import *

def tune():
    with open("kernel_tuner/examples/cuda/convolution.cu", "r") as f:
        kernel_string = f.read()

    filter_size = (17, 17)
    problem_size = (4096, 4096)
    size = numpy.prod(problem_size)
    border_size = (filter_size[0] // 2 * 2, filter_size[1] // 2 * 2)
    input_size = (problem_size[0] + border_size[0]) * (problem_size[1] + border_size[1])

    output = numpy.zeros(size).astype(numpy.float32)
    input = numpy.random.randn(input_size).astype(numpy.float32)

    filter = numpy.random.randn(filter_size[0] * filter_size[1]).astype(numpy.float32)
    cmem_args = {"d_filter": filter}

    args = [output, input, filter]
    tune_params = OrderedDict()
    tune_params["filter_width"] = [filter_size[0]]
    tune_params["filter_height"] = [filter_size[1]]

    # tune_params["block_size_x"] = [16*i for i in range(1,3)]
    tune_params["block_size_x"] = [16 * i for i in range(1, 9)]
    # tune_params["block_size_y"] = [2**i for i in range(1,5)]
    tune_params["block_size_y"] = [2**i for i in range(1, 6)]

    tune_params["tile_size_x"] = [2**i for i in range(3)]
    tune_params["tile_size_y"] = [2**i for i in range(3)]

    tune_params["use_padding"] = [
        0,
        1,
    ]  # toggle the insertion of padding in shared memory
    tune_params["read_only"] = [0, 1]  # toggle using the read-only cache

    grid_div_x = ["block_size_x", "tile_size_x"]
    grid_div_y = ["block_size_y", "tile_size_y"]

    # compute the answer using a naive kernel
    params = {"block_size_x": 16, "block_size_y": 16}
    tune_params["filter_width"] = [filter_size[0]]
    tune_params["filter_height"] = [filter_size[1]]
    results = kernel_tuner.run_kernel(
        "convolution_naive",
        kernel_string,
        problem_size,
        args,
        params,
        grid_div_y=["block_size_y"],
        grid_div_x=["block_size_x"],
        lang="cupy",
    )

    # set non-output fields to None
    answer = [results[0], None, None]

    # start kernel tuning with correctness verification
    return kernel_tuner.tune_kernel(
        "convolution_kernel",
        kernel_string,
        problem_size,
        args,
        tune_params,
        grid_div_y=grid_div_y,
        grid_div_x=grid_div_x,
        verbose=True,
        cmem_args=cmem_args,
        answer=answer,
        lang="cupy",
        strategy="genetic_algorithm",
        strategy_options=dict(max_fevals=5)
    )


if __name__ == "__main__":
    import time

    s1 = time.time() * 1000
    results = tune()
    print(results)

    e1 = time.time() * 1000
    print("\n Actual time used:", e1 - s1)
    import json

    with open("convolution_gpu_runtime.json", "w") as fp:
        json.dump(results, fp)

Using: Tesla T4
Using: Tesla T4
filter_width=17, filter_height=17, block_size_x=32, block_size_y=8, tile_size_x=1, tile_size_y=2, use_padding=0, read_only=1, time=8.863ms
filter_width=17, filter_height=17, block_size_x=64, block_size_y=16, tile_size_x=1, tile_size_y=4, use_padding=1, read_only=1, time=13.511ms
filter_width=17, filter_height=17, block_size_x=80, block_size_y=4, tile_size_x=1, tile_size_y=4, use_padding=1, read_only=0, time=5.722ms
filter_width=17, filter_height=17, block_size_x=16, block_size_y=16, tile_size_x=1, tile_size_y=1, use_padding=1, read_only=1, time=14.330ms
filter_width=17, filter_height=17, block_size_x=48, block_size_y=4, tile_size_x=1, tile_size_y=4, use_padding=1, read_only=1, time=5.216ms
max_fevals reached
best performing configuration:
filter_width=17, filter_height=17, block_size_x=48, block_size_y=4, tile_size_x=1, tile_size_y=4, use_padding=1, read_only=1, time=5.216ms
([OrderedDict([('filter_width', 17), ('filter_height', 17), ('block_size_x', 32)

### Custom Strategy

This is where we will be tuning our reinforcement learning strategy to optimize the demo kernel.

Demo Kernel 1: Vector Add



In [5]:
# default imports to get started
from kernel_tuner import util
from kernel_tuner.searchspace import Searchspace
from kernel_tuner.strategies import common
from kernel_tuner.strategies.common import CostFunc

# NOTE: Q-learning strategy